## 데이터 불러오기

In [81]:
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline
# 랜덤에 의해 똑같은 결과를 재현하도록 시드 설정
# 하이퍼파라미터를 튜닝하기 위한 용도(흔들리면 무엇때문에 좋아졌는지 알기 어려움)
tf.set_random_seed(777)

In [82]:
import requests
url = 'https://biendata.com/competition/airquality/bj/2018-03-01-0/2018-05-10-23/2k0d1d8'
respones= requests.get(url)
with open ("bj_airquality.csv",'w') as f:
    f.write(respones.text)

In [83]:
import pandas as pd
b_air = pd.read_csv("bj_airquality.csv")

In [84]:
b_air.columns = ['id', 'test_id', 'time', 'PM2.5', 'PM10','NO2', 'CO', 'O3', 'SO2']

In [85]:
sample = pd.read_csv("sample_submission.csv")

In [86]:
def function1(i, y):
    dataset = b_air[b_air["test_id"] == i][y]
    return dataset

In [87]:
mean = np.mean(pd.DataFrame(function1('nansanhuan_aq','PM2.5')))
aq1 = list(pd.DataFrame(function1('nansanhuan_aq','PM2.5')).fillna(round(mean,1))['PM2.5'])

In [88]:
def min_max_scaling(x):
    x_np = np.asarray(x)
    return (x_np-x_np.min()) / (x_np.max()-x_np.min() + 1e-7)

In [89]:
def reverse_scaling(org_x, x):
    org_x_np = np.asarray(org_x)
    x_np = np.asarray(x)
    return (x_np*(org_x_np.max()-org_x_np.min() + 1e-7)) + org_x_np.min()

In [34]:
#round(reverse_scaling(aq1,aq)[82],1)

In [93]:
aq_reverse = aq[::-1]

In [95]:
data=[]
for i in range(len(aq)-48):
    a = aq[i:i+48]
    data.append(a)

In [63]:
data_reverse

[[0.2506887051650995,
  0.2644628098445006,
  0.25895316797274015,
  0.31404958669034444,
  0.35261707979266743,
  0.38292011008734983,
  0.4187327822537926,
  0.44903581254847497,
  0.5151515150096001,
  0.5151515150096001,
  0.4986225893943188,
  0.4462809916125947,
  0.3911845728949905,
  0.3966942147667509,
  0.5123966940737199,
  0.6501377408677306,
  0.7603305783029392,
  0.7796143248541006,
  0.7685950411105799,
  0.7685950411105799,
  0.7052341595853349,
  0.3966942147667509,
  0.2892561982674225,
  0.15977961428105245,
  0.192837465511615,
  0.3030303029468236,
  0.38292011008734983,
  0.45179063348435516,
  0.3250688704338653,
  0.3250688704338653,
  0.3250688704338653,
  0.3553719007285477,
  0.6170798896371681,
  0.7685950411105799,
  0.7933884295335017,
  0.8154269970205434,
  0.8567493110587467,
  0.8457300273152258,
  0.8126721760846632,
  0.7741046829823403,
  0.7134986223929756,
  0.6639118455471317,
  0.6198347105730483,
  0.6308539943165691,
  0.6611570246112515,
  0

In [64]:
data_X=[]
data_Y=[]
seq_length = 7
for i in range(len(data)-seq_length):
    data_X.append(data[i:i+seq_length]) # i+seq_length번째 날 예측
    data_Y.append(data[i+seq_length])

In [67]:
train_size = int(len(data_Y) * 0.8)
test_size = len(data_Y) - train_size
 
# 데이터를 잘라 학습용 데이터 생성
trainX = data_X[:train_size]
trainY = data_Y[:train_size]
 
# 데이터를 잘라 테스트용 데이터 생성
testX = data_X[train_size:]
testY = data_Y[train_size:]

In [71]:
len(trainX[0][0])

48

In [72]:
trainX = np.reshape(trainX, [len(trainX),seq_length,48])
testX = np.reshape(testX, [len(testX),seq_length,48])
trainY = np.reshape(trainY,[len(trainY),48])
testY = np.reshape(testY,[len(testY),48])

## model

In [73]:
import tensorflow as tf
X = tf.placeholder(tf.float32, [None, seq_length, 48])
Y = tf.placeholder(tf.float32, [None, 48])

targets = tf.placeholder(tf.float32, [None, 48])
predictions = tf.placeholder(tf.float32, [None, 48])

In [74]:
# 하이퍼파라미터
rnn_cell_hidden_dim = 32   # 각 셀의 (hidden)출력 크기
forget_bias = 1.0          # 망각편향(기본값 1.0)
num_stacked_layers = 3     # stacked LSTM layers 개수
keep_prob = 1.0            # dropout할 때 keep할 비율
 
epoch_num = 1000           # 에폭 횟수(학습용전체데이터를 몇 회 반복해서 학습할 것인가 입력)
learning_rate = 0.01       # 학습률

In [75]:
def lstm_cell():
    cell = tf.contrib.rnn.BasicLSTMCell(num_units=rnn_cell_hidden_dim, 
                                        forget_bias=forget_bias, state_is_tuple=True, activation=tf.nn.softsign)
    if keep_prob < 1.0:
        cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
    return cell
 
# num_stacked_layers개의 층으로 쌓인 Stacked RNNs 생성
stackedRNNs = [lstm_cell() for _ in range(num_stacked_layers)]
multi_cells = tf.contrib.rnn.MultiRNNCell(stackedRNNs, state_is_tuple=True) if num_stacked_layers > 1 else lstm_cell()
 
# RNN Cell(여기서는 LSTM셀임)들을 연결
hypothesis, _states = tf.nn.dynamic_rnn(multi_cells, X, dtype=tf.float32)
print("hypothesis: ", hypothesis)
 
hypothesis = tf.contrib.layers.fully_connected(hypothesis[:, -1], 48, activation_fn=tf.nn.relu)

hypothesis:  Tensor("rnn/transpose_1:0", shape=(?, 7, 32), dtype=float32)


In [76]:
loss = tf.reduce_sum(tf.square(hypothesis - Y))
# 최적화함수로 AdamOptimizer를 사용한다
optimizer = tf.train.AdamOptimizer(learning_rate)
# optimizer = tf.train.RMSPropOptimizer(learning_rate) # LSTM과 궁합 별로임
 
train = optimizer.minimize(loss)
 
# RMSE(Root Mean Square Error)
# 제곱오차의 평균을 구하고 다시 제곱근을 구하면 평균 오차가 나온다
# rmse = tf.sqrt(tf.reduce_mean(tf.square(targets-predictions))) # 아래 코드와 같다
rmse = tf.sqrt(tf.reduce_mean(tf.squared_difference(targets, predictions)))

temp = tf.divide(tf.abs(targets-predictions, targets+predictions)
smape = 2*tf.reduce_mean(temp)

In [77]:
def smape(targets, predictions):
    dividend= np.abs(np.array(targets) - np.array(predictions))
    denominator = np.array(targets) + np.array(predictions)
    
    return 2 * np.mean(np.divide(dividend, denominator, out=np.zeros_like(dividend), where=denominator!=0, casting='unsafe'))

In [78]:
trainY.shape

(692, 48)

In [79]:
tf.reset_default_graph()

In [80]:
train_error_summary = [] # 학습용 데이터의 오류를 중간 중간 기록한다
test_error_summary = []  # 테스트용 데이터의 오류를 중간 중간 기록한다
test_predict = ''        # 테스트용데이터로 예측한 결과
 
sess = tf.Session()
sess.run(tf.global_variables_initializer())
 
# 학습한다
print('학습을 시작합니다...')

for epoch in range(epoch_num):
    _, _loss = sess.run([train, loss], feed_dict={X: trainX, Y: trainY})
    if ((epoch+1) % 100 == 0) or (epoch == epoch_num-1): # 100번째마다 또는 마지막 epoch인 경우
        
        # 학습용데이터로 rmse오차를 구한다
        train_predict = sess.run(hypothesis, feed_dict={X: trainX})
        train_error = sess.run(rmse, feed_dict={targets: trainY, predictions: train_predict})
        train_error_summary.append(train_error)
 
        # 테스트용데이터로 rmse오차를 구한다
        test_predict = sess.run(hypothesis, feed_dict={X: testX})
        test_error = sess.run(rmse, feed_dict={targets: testY, predictions: test_predict})
        test_error_summary.append(test_error)
        
        # 현재 오류를 출력한다
        print("epoch: {}, train_error(A): {}, test_error(B): {}, B-A: {}".format(epoch+1, train_error, test_error, test_error-train_error))

학습을 시작합니다...


TypeError: Cannot interpret feed_dict key as Tensor: Tensor Tensor("Placeholder:0", shape=(?, 7, 48), dtype=float32) is not an element of this graph.

## 예측

In [65]:
recent_data = np.reshape(data_X[len(data_X)-seq_length:][0], [-1,seq_length,48])

In [69]:
test_predict = sess.run(hypothesis, feed_dict={X: recent_data})
print("test_predict", test_predict[0])

test_predict [63.494213 62.681168 61.92765  60.333015 58.163963 56.24036  55.01711
 53.942757 53.36801  53.193657 53.8173   53.927723  0.       54.172485
 55.046227 56.170197 57.619366 59.914955 61.237022 61.7101   61.56108
 61.584167 61.411358 61.162594 60.835262 59.939972 58.491993 57.44391
 56.020782 54.97223  54.347515 53.89742  53.69812  53.472897 53.212986
 53.003155 53.177715 50.77866  50.678593 50.45371   0.       50.32879
 49.803925 49.028954 48.42899  47.85403  47.454002 48.728935]


In [67]:
def smape(actual, predicted):
    dividend= np.abs(np.array(actual) - np.array(predicted))
    denominator = np.array(actual) + np.array(predicted)
    
    return 2 * np.mean(np.divide(dividend, denominator, out=np.zeros_like(dividend), where=denominator!=0, casting='unsafe'))

In [68]:
smape(data_Y[49], test_predict)

0.5023441588366646